In [1]:
import seaborn as sns
# from sklearn.model_selection import train_test_split
import gc
import os
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from collections import Counter
import matplotlib.pyplot as plt

%matplotlib inline
#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [26]:
result = pd.read_csv('./submit/submission_test2_0.75.csv')

result.shape
result.head()

(2265879, 3)

,aid,uid,score
0,1790,55374290,0.009278
1,1918,30704418,0.043932
2,1904,44454952,0.045259
3,562,30747797,0.050961
4,886,19138638,0.019318


In [27]:
result.score.mean()

0.05042170168964725

In [28]:
# noise = np.random.rand(result.shape[0]) / 10
noise = np.random.normal(loc=0.05, scale=0.05, size=result.shape[0])

In [29]:
# result.score[:5]
result.score = result.score * 0.95 + noise * 0.05
# result.score[:5]

0    0.009278
1    0.043932
2    0.045259
3    0.050961
4    0.019318
Name: score, dtype: float64

0    0.011516
1    0.045723
2    0.046714
3    0.049851
4    0.021742
Name: score, dtype: float64

In [31]:
sub_zip_name = 'lgb005_new'
result['score'] = result['score'].apply(lambda x: float('%.8f' % x))
result.to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')

0

0

0

## lgb

In [2]:
import lightgbm as lgb

In [3]:
df = pd.read_csv('df_b.csv')

df.shape

(11064803, 156)

In [4]:
df['label'][df.label == -1] = 0

train = df[df.label != -99]
test = df[df.label == -99]

train.shape
test.shape

del df
gc.collect()

(8798814, 156)

(2265989, 156)

18

In [5]:
by = ['aid', 'uid']
target = ['label']
predictors = list(set(train.columns) - set(by) - set(target))

In [ ]:
# lgb_cv
params = {
    'objective':'binary',
        # metric='binary_error',
        'num_leaves':7,
        'max_depth':3,
        'learning_rate':0.05,
        'reg_alpha' : .4,
        'reg_lambda' : .2,
        'random_state':1024,
        'colsample_bytree':0.8,
        'subsample':0.9,
#         'n_estimators':20000,
        'n_jobs': 31,
#         'device':'gpu',
        'histogram_pool_size':30270,
        'max_bin':63,
        'gpu_use_dp':True
}
        
len(predictors)

lgb_train = lgb.Dataset(train[predictors],label = train['label'])
print('Start training')
cv = lgb.cv(params,lgb_train,num_boost_round=10000,nfold=3,early_stopping_rounds=50,seed=1024,verbose_eval=100)

print('Baseline->' + str(len(cv['binary_logloss-mean'])) + ':' + str(cv['binary_logloss-mean'][-1]))

In [14]:
lgb_online = lgb.LGBMClassifier(
        objective='binary',
        metric='binary_logloss',
        num_leaves=7,#35,
        max_depth=3,
        learning_rate=0.05, # small_eta
        reg_alpha = .4,
        reg_lambda = .2,
        colsample_bytree = .8,
        subsample = .9,
        random_state=1024,
        n_estimators=4000,
        n_jobs = 31,
        histogram_pool_size=30270,
        max_bin=63
    )
len(predictors)

# submit_model = lgb_online.fit(train[predictors], train['label'])

test['score'] = submit_model.predict_proba(test[predictors])[:,1]
test['score'] = test['score'].apply(lambda x: float('%.8f' % x))
sub_zip_name = 'lgb'
test[['aid', 'uid','score']].to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')

153

0

0

0

In [8]:
test1 = pd.read_csv('test_kflod_b0')
test2 = pd.read_csv('test_kflod_b1')
test3 = pd.read_csv('test_kflod_b2')
test4 = pd.read_csv('test_kflod_b3')
test5 = pd.read_csv('test_kflod_b4')

test1.shape
test2.shape
test3.shape
test4.shape
test5.shape

(2265879, 172)

(2265879, 172)

(2265879, 172)

(2265879, 172)

(2265879, 172)

In [13]:
test = test1 + test2 + test3 + test4 + test5
test = test[predictors + by]
test = test / 5

test.shape

# test1.head()
# test.head()

(2265879, 155)

In [ ]:
del test1
del test2
del test3
del test4
del test5
gc.collect()